In [1]:
import streamlit as st
import openai
import json
import ast
import pandas as pd

In [2]:
openai_api_key = ""
openai.api_key = openai_api_key

In [3]:
# Path to the JSON file
file_path = "report.json"

# Open and read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

In [1]:
# data['sections']

In [29]:
config_data = data['sections'][0]['visualContainers'][27]['config']
config_data = json.loads(config_data)

In [41]:
config_data['singleVisual']['prototypeQuery']['Select'][0]['NativeReferenceName']

'Boutique Name'

In [17]:
# config_data

In [4]:
dict_page_slicers = {}

for section in data.get("sections", []):
    # only include the pages which are not hidden in the dashboard
    page_config = json.loads(section['config'])
    if page_config.get('visibility') != 1:
        # slicer_list_per_page = []
        slicer_list_per_page = {}
        # Store displayName to know the section/page name
        section_summary = {
            "displayName": section.get("displayName", ""),
            "visualContainers": []
        }

        # Process each visual container in the section
        for i in range(len(section.get("visualContainers", []))):
            visual = section.get("visualContainers", [])[i]
            # Parse config if it's a string
            config_data = visual.get("config", {})
            if isinstance(config_data, str):
                config_data = json.loads(config_data)

                visual_type_to_be_included = ['slicer']
                if config_data.get("singleVisual", {}).get("visualType", "") in visual_type_to_be_included:
                    visual_type = config_data.get("singleVisual", {}).get("visualType", "")
                    if 'NativeReferenceName' in config_data['singleVisual']['prototypeQuery']['Select'][0]:
                        slicer_name = config_data['singleVisual']['prototypeQuery']['Select'][0]['NativeReferenceName']
                    else: slicer_name = config_data['singleVisual']['prototypeQuery']['Select'][0]['Name']
                    # slicer_list_per_page.append(slicer_name)
                    slicer_list_per_page[slicer_name] = visual_type

        dict_page_slicers[section.get("displayName", "")] = slicer_list_per_page

# List to store each row as a dictionary
rows = []

# Loop through each page and its visuals
for page_name, visuals in dict_page_slicers.items():
    for visual_name, visual_type in visuals.items():
        # Append a dictionary for each visual
        rows.append({
            "page name": page_name,
            "visual name": visual_name,
            "visual type": visual_type
        })

# Convert list of dictionaries into a DataFrame
df = pd.DataFrame(rows)

# Display the DataFrame
print(df)

       page name         visual name visual type
0       Scope FA       Boutique Name      slicer
1       Scope FA                Date      slicer
2       Scope FA  InchanelBtq_Status      slicer
3       Scope FA             Country      slicer
4        Digital                Date      slicer
5        Digital             Country      slicer
6        Digital       Boutique Name      slicer
7        Digital           Date.Date      slicer
8   Client Focus                Date      slicer
9   Client Focus           Date.Date      slicer
10  Client Focus       Boutique Name      slicer
11  Client Focus             Country      slicer
12        Mirror                Date      slicer
13        Mirror       Boutique Name      slicer
14        Mirror           Date.Date      slicer
15        Mirror             Country      slicer


In [7]:
function_descriptions = [
    {
        "name": "summary_in_target_platform",
        "description": "Generate documentation for a Power BI report tailored to a specified platform and language.",
        "parameters": {
            "type": "object",
            "properties": {
                "language": {
                    "type": "string",
                    "description": "The language in which the documentation should be written (e.g., English, French)."
                },
                "platform": {
                    "type": "string",
                    "description": "The target platform where the documentation will be used (e.g., Confluence, SharePoint)."
                }
            }
        }
    },
    {
        "name": "uniformize_visuals",
        "description": "This function automates the standardization of visual styles across different pages of a Power BI dashboard. It updates the formatting of specified target visuals to match the style of a designated source visual. The function ensures consistent visual presentation, aiding in seamless user experiences across various dashboard pages.",
        "parameters": {
            "type": "object",
            "properties": {
                "source": {
                    "type": "object",
                    "properties": {
                        "source_page": {
                            "type": "string",
                            "description": "Specifies the page name where the source visual is located. This value must correspond to the 'page name' column of the dataframe. Only one source page is allowed."
                        },
                        "source_visual": {
                            "type": "string",
                            "description": "The name of the visual on the source page used as the formatting template. This should be extracted from the 'visual name' column of the dataframe. Only one source visual is permitted."
                        }
                    }
                },
                "target": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "target_page": {
                                "type": "string",
                                "description": "The name of the page containing the target visuals that will be updated. This is taken from the 'page name' column of the dataframe. Multiple target pages can be specified."
                            },
                            "target_visual": {
                                "type": "string",
                                "description": "The identifier of each target visual on the target page to be formatted. These identifiers are sourced from the 'visual name' column of the dataframe. Multiple target visuals can be specified per page."
                            }
                        }
                    }
                }
            }
        }
    }
]

In [9]:
def generate_completion(user_input):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": user_input}],
        functions=function_descriptions,
        function_call="auto",  # Let the model decide if it needs to call the function
    )
    arguments = completion.choices[0].message["function_call"]["arguments"]
    return arguments
    # return completion.choices[0].message if completion.choices else {}

In [16]:
# user_prompt1 = (
#     "I want to update all the slicers on the 'Digital' and 'Client Focus' pages so that their date slicers match the format of the 'Date' slicer on the 'Scope FA' page. "
# )

user_prompt2 = (
    "I want to update the slicers all the pages in the dashboard so that their format match the 'Date' slicer on the 'Scope FA' page. "
)

total_prompt = (
    "You are an assistant designed to identify the source page, source visuals, target page, and target visuals based on user input. "
    "Your answer should be based on the given DataFrame, which contains information about all the pages and visuals in the dashboard. "
    "It is essential to understand the user's intention and ensure that nothing in the DataFrame is omitted. "
    "For example, if the user does not specify a particular page for uniformization, they might want to apply uniformization to all the pages.\n"
    f"Here is the user input: {user_prompt2}\n"
    f"Here is the DataFrame you should base your analysis on:\n{df}"
)

output = generate_completion(total_prompt)
print(output)
# print(output.get("function_call", {}).get("name"))

{"source":{"source_page":"Scope FA","source_visual":"Date"},"target":[{"target_page":"Scope FA","target_visual":"Boutique Name"},{"target_page":"Scope FA","target_visual":"InchanelBtq_Status"},{"target_page":"Scope FA","target_visual":"Country"},{"target_page":"Digital","target_visual":"Date"},{"target_page":"Digital","target_visual":"Country"},{"target_page":"Digital","target_visual":"Boutique Name"},{"target_page":"Digital","target_visual":"Date.Date"},{"target_page":"Client Focus","target_visual":"Date"},{"target_page":"Client Focus","target_visual":"Date.Date"},{"target_page":"Client Focus","target_visual":"Boutique Name"},{"target_page":"Client Focus","target_visual":"Country"},{"target_page":"Mirror","target_visual":"Date"},{"target_page":"Mirror","target_visual":"Boutique Name"},{"target_page":"Mirror","target_visual":"Date.Date"},{"target_page":"Mirror","target_visual":"Country"}]}
